<a href="https://colab.research.google.com/github/hublun/TWSApi/blob/master/IBApi_Historical_Data_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [287]:
import pandas as pd
import scipy.stats as stat
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_pickle('/content/drive/MyDrive/TWS/M2K20201130_1204.pkl')

In [ ]:
df["Time"] = pd.to_datetime(df["time"], format='%Y%m%d  %H:%M:%S', yearfirst=True)

In [ ]:
df.drop('time', axis=1, inplace=True)

In [ ]:
df["Hour"] = pd.DatetimeIndex(df["Time"]).hour
df["Minute"] = pd.DatetimeIndex(df["Time"]).minute
df["Second"] = pd.DatetimeIndex(df["Time"]).second
df["Day"] = pd.DatetimeIndex(df["Time"]).day

In [ ]:
initialDateTime = df.Time[0]
initialDateTime

Timestamp('2020-11-29 18:00:00')

In [ ]:
df["TradeDay"] = pd.TimedeltaIndex(df.Time - initialDateTime).days

In [ ]:
df.head()

In [ ]:
df["mu10min"] = df.avg.rolling(window=120, min_periods=1).mean()

df["mu5min"] = df.avg.rolling(window=60, min_periods=1).mean()
df["mu2min"] = df.avg.rolling(window=24, min_periods=1).mean()
df["mumin"] = df.avg.rolling(window=12, min_periods=1).mean()
df["dev"] = df.close - df.open

df["delta1m"] = df.avg - df.mumin
df["delta2m"] = df.avg - df.mu2min

df["delta5m"] = df.avg - df.mu5min
df["delta10m"] = df.avg - df.mu10min
 

In [ ]:
df.head()

In [ ]:

df["pbmin"] = (df.avg - df.avg.shift(periods=12)).fillna(0.0)
df["pb5min"] = (df.avg - df.avg.shift(periods=5*12)).fillna(0.0)
df["pb10min"] = (df.avg - df.avg.shift(periods=10*12)).fillna(0.0)
df["pb15min"] = (df.avg - df.avg.shift(periods=15*12)).fillna(0.0)
df["pb30min"] = (df.avg - df.avg.shift(periods=30*12)).fillna(0.0)
df.head()

In [ ]:
df["pfmin"] = (df.avg.shift(-12)-df.avg).fillna(0.0)
df["pf2min"] = (df.avg.shift(-2*12)-df.avg).fillna(0.0)
df["pf5min"] = (df.avg.shift(-5*12)-df.avg).fillna(0.0)
df["pf10min"] = (df.avg.shift(-10*12)-df.avg).fillna(0.0)
df["pf15min"] = (df.avg.shift(-15*12)-df.avg).fillna(0.0)
df.tail()

In [ ]:
a4_dims = (23,12)
fig, ax = plt.subplots(figsize=a4_dims)
sns.scatterplot(x=df.pfmin, y=df.delta1m)

In [ ]:
df.columns[11:33]

In [ ]:
df.drop(df.head(30*12).index, inplace=True)
df.drop(df.tail(15*12).index, inplace=True)

In [ ]:
df.shape[0]

80947

In [ ]:
df_corr = df[df.columns[11:33]]
 

In [ ]:
df_corr.head()

In [ ]:
df_corr.corr()

In [283]:
import statsmodels.formula.api as smf
lm = smf.ols(formula="pfmin~dev+delta5m+delta10m", data=df).fit()
lm.params
lm.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                  Results: Ordinary least squares
====================================================================
Model:              OLS              Adj. R-squared:     0.009      
Dependent Variable: pfmin            AIC:                151730.0499
Date:               2020-12-05 22:29 BIC:                151767.2561
No. Observations:   80947            Log-Likelihood:     -75861.    
Df Model:           3                F-statistic:        246.2      
Df Residuals:       80943            Prob (F-statistic): 4.53e-159  
R-squared:          0.009            Scale:              0.38156    
----------------------------------------------------------------------
              Coef.    Std.Err.      t      P>|t|     [0.025    0.975]
----------------------------------------------------------------------
Intercept     0.0057     0.0022    2.6144   0.0089    0.0014    0.0099
dev           0.4088     0.0154   26.5700   0.0000    0.3786    0.4389
delta5m       0.0089     0.0058    1.5265   0.1269   -0.0025    0.0204
delta10m     -0.0187     0.0041   -4.5487   0.0000   -0.0268   -0.0107
--------------------------------------------------------------------
Omnibus:            11601.955      Durbin-Watson:         0.167     
Prob(Omnibus):      0.000          Jarque-Bera (JB):      152837.613
Skew:               -0.217         Prob(JB):              0.000     
Kurtosis:           9.718          Condition No.:         9         
====================================================================

"""

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split

In [ ]:
df.columns[12]

'pf15min'

In [ ]:
df.drop(df.columns[0],axis=1, inplace=True)

In [ ]:
df.reset_index(inplace=True)
label = df.pfmin
data = df[df.columns[:-1]]

In [ ]:
x=data.values
y=label.values


In [284]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 1
}

x, x_test, y, y_test = train_test_split(x, y, test_size= 0.2, random_state=42)
test_data = lgb.Dataset(x_test, y_test)

model = lgb.train(params, train_set=train_dataset, valid_sets=test_data, verbose_eval=True)

[1]	valid_0's l1: 0.418289	valid_0's l2: 0.402081
[2]	valid_0's l1: 0.417762	valid_0's l2: 0.4007
[3]	valid_0's l1: 0.417365	valid_0's l2: 0.399542
[4]	valid_0's l1: 0.416848	valid_0's l2: 0.398041
[5]	valid_0's l1: 0.416525	valid_0's l2: 0.396648
[6]	valid_0's l1: 0.41613	valid_0's l2: 0.395443
[7]	valid_0's l1: 0.415805	valid_0's l2: 0.394371
[8]	valid_0's l1: 0.415505	valid_0's l2: 0.393437
[9]	valid_0's l1: 0.415191	valid_0's l2: 0.392133
[10]	valid_0's l1: 0.414949	valid_0's l2: 0.391057
[11]	valid_0's l1: 0.414621	valid_0's l2: 0.389843
[12]	valid_0's l1: 0.414332	valid_0's l2: 0.388792
[13]	valid_0's l1: 0.41406	valid_0's l2: 0.387937
[14]	valid_0's l1: 0.413835	valid_0's l2: 0.387139
[15]	valid_0's l1: 0.413653	valid_0's l2: 0.386486
[16]	valid_0's l1: 0.413279	valid_0's l2: 0.38499
[17]	valid_0's l1: 0.413005	valid_0's l2: 0.383985
[18]	valid_0's l1: 0.412748	valid_0's l2: 0.383245
[19]	valid_0's l1: 0.412449	valid_0's l2: 0.38206
[20]	valid_0's l1: 0.412156	valid_0's l2: 0.38

In [288]:
model.save_model("lgm_20201205.lgm")

In [ ]:
"{0:.2f}%".format(100*df.pfmin[ abs(df.pfmin)< 1].shape[0]/df.shape[0])

'94.24%'

In [ ]:
fig, ax = plt.subplots(figsize=a4_dims)
sns.kdeplot(df.mu10min, shade=True, vertical=True, color="red")